# 주의
## 1. 크롤링하는 화면 절대 클릭 금지 -> 중간에 '저작권 어쩌구'로 멈춘다.
## 2. 이상시 연락..주지 말고 구글링해보고 안되면 연락...
##### 3. 창이 여러 개 떠도 그러려니


In [136]:
import selenium
from selenium import webdriver as wd
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import csv

# 자신의 드라이버 경로 지정
driver = wd.Chrome(executable_path=r'...')
time.sleep(2)

# 윈도우 크기 조절: 화면이 작으면 로그인 창이 활성화되지 않음
driver.maximize_window()

# 홈페이지 부르기
driver.get('https://brunch.co.kr/')

In [137]:
search_box = driver.find_element_by_xpath('//*[@id="btnServiceMenuSearch"]')
search_box.click()

In [138]:
search_bar = driver.find_element_by_xpath('//*[@id="txt_search"]')

In [139]:
keyword = '슬픈' ## 크롤링할 키워드 쓰기.
search_bar.send_keys(keyword)
search_bar.send_keys(Keys.RETURN)

In [140]:
SCROLL_PAUSE_TIME= 3
start_page = 1
diary = []
total_num = 0

last_height = driver.execute_script("return document.body.scrollHeight")
cnt = 0
#while cnt < 100:
while True:    
    cnt += 1
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    print("brunch-{} ~ {}".format(start_page, start_page+19))
    
    for page in range(start_page, start_page+19):
        try:
            acting_point = driver.find_element_by_xpath(
                f'//*[@id="resultArticle"]/div/div[1]/div[2]/ul/li[{page}]/a/div[1]/strong')
            driver.execute_script("arguments[0].click();", acting_point)
        except:
            pass
        driver.switch_to.window(driver.window_handles[1])
        driver.get_window_position(driver.window_handles[1])
        driver.implicitly_wait(2)
        diary_content = ''
        try:
            div_p_elems = driver.find_elements_by_xpath(
                '/html/body/div[3]/div[1]/div[2]/div[1]/p[@class="wrap_item item_type_text"] | /html/body/div[3]/div[1]/div[2]/div[1]/h4[@class="wrap_item item_type_text"]')
            for item in div_p_elems:
                diary_content += item.text
            diary_content = diary_content.replace('\n', "")
        except:
            pass

        a = driver.page_source
        bs = BeautifulSoup(a, "html.parser")
        c = bs.find_all('div', {'class': 'wrap_img_float'})
        diary_img = []
        
        for i in c:
            x = re.findall(r'(?=src)src=\"(?P<src>[^\"]+)', str(i))
            diary_img += x

        diary_title = ''
        try:
            diary_title = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[1]/div/div[3]/h1')
            diary_title = diary_title.text.replace('\n', "")
        except:
            pass
        
        sub_title = ''
        try:
            sub_title = driver.find_element_by_xpath('//html/body/div[3]/div[1]/div[1]/div/div[3]/p')
            sub_title = sub_title.text.replace('\n', "")
        except:
            pass
        
        list_keyword = ''
        try:
            list_keyword = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div[2]/div/div/ul')
            list_keyword = list_keyword.text.replace('\n', "/")
        except:
            pass

        diary_date = ''
        try:
            diary_date = driver.find_element_by_xpath('//*[@id="wrapArticleInfo"]/span[4]')
        except:
            pass

        time.sleep(SCROLL_PAUSE_TIME)

        diary.append({
            "title": diary_title,
            "subtitle": sub_title,
            "date": diary_date.text,
            "content": diary_content,
            "list_keyword": list_keyword,
            "img": diary_img,
        })
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.get_window_position(driver.window_handles[0])

    start_page += 19
    if cnt % 10 == 0:
        a = pd.DataFrame(diary)
        a = a[['title', 'subtitle', 'date', 'content', 'list_keyword']]
        a = a.drop_duplicates()#.sum()
        total_num = total_num + len(a)
        name_k = keyword + '_{}.csv'.format(cnt)
        a.to_csv(name_k, encoding='utf-8-sig')
        diary = []
        print('-------------------현재 총 개수:', total_num, '-------------')
    if total_num > 500: ### 크롤링하고 싶은 데이터 개수
        break

brunch-1 ~ 20


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")


brunch-20 ~ 39
brunch-39 ~ 58
brunch-58 ~ 77
brunch-77 ~ 96
brunch-96 ~ 115
brunch-115 ~ 134
brunch-134 ~ 153
brunch-153 ~ 172
brunch-172 ~ 191
-------------------현재 총 개수: 155 -------------
brunch-191 ~ 210
brunch-210 ~ 229
brunch-229 ~ 248
brunch-248 ~ 267
brunch-267 ~ 286
brunch-286 ~ 305
brunch-305 ~ 324
brunch-324 ~ 343
brunch-343 ~ 362
brunch-362 ~ 381
-------------------현재 총 개수: 278 -------------
brunch-381 ~ 400
brunch-400 ~ 419
brunch-419 ~ 438
brunch-438 ~ 457
brunch-457 ~ 476
brunch-476 ~ 495
brunch-495 ~ 514
brunch-514 ~ 533
brunch-533 ~ 552
brunch-552 ~ 571
-------------------현재 총 개수: 387 -------------
brunch-571 ~ 590
brunch-590 ~ 609
brunch-609 ~ 628
brunch-628 ~ 647


UnexpectedAlertPresentException: Alert Text: 저작권 보호를 위하여 브런치 작가 본인만 글을 복사할 수 있습니다. 너른 양해 부탁드립니다.
Message: unexpected alert open: {Alert text : 저작권 보호를 위하여 브런치 작가 본인만 글을 복사할 수 있습니다. 너른 양해 부탁드립니다.}
  (Session info: chrome=92.0.4515.159)


In [142]:
driver.quit()

In [143]:
# 저장된 파일들 불러오기
sad_10 = pd.read_csv('슬픈_10.csv')
sad_10.drop('Unnamed: 0', axis=1, inplace=True)

sad_20 = pd.read_csv('슬픈_20.csv')
sad_20.drop('Unnamed: 0', axis=1, inplace=True)

sad_30 = pd.read_csv('슬픈_30.csv')
sad_30.drop('Unnamed: 0', axis=1, inplace=True)
sad_30

,title,subtitle,date,content,list_keyword
0,영원불멸한 관계의 슬픈 미학,자기 안에 우주를 담고 있는 모든 존재들에게,Dec 15. 2020,어느 날 내 주위의 모든 것들이 무너지는 듯한 때가 누구에게나 한 번쯤은 있다.끝이...,우주/인생/세상
1,"이름이 없어 슬픈 그대에게,",누구나 자기의 자리가 있다.,Sep 10. 2020,나는 이름 없는 작가다. 200여 건의 입고 메일을 돌리고 이름 없는 내 책을 ...,"09화/입고 메일 돌리기/10화/이름이 없어 슬픈 그대에게,/현재글/11화/작가라는..."
2,"조두순 출소, 유튜버들의 돈벌이 대잔치 되다.",양심과 도덕성을 잃어버린 슬픈 현실,Dec 15. 2020,지난 12일(토) 조두순이 모두의 걱정과 염려 속에 결국 출소를 하였다. 그는 20...,사회/성범죄/성폭력
3,광석이형이 이야기하는 슬픈노래,슬픈노래,Nov 15. 2020,잔잔한 일요일 아침입니다. 가을비 내려 단풍잎이 떨어지기 전에 가을을 맘껏 즐겨야겠...,11화/김광석이라는 악기/12화/광석이형이 이야기하는 슬픈노래/현재글/13화/쉰즈음...
4,견딜 수 없이 슬픈,"Sia, ‘Snowman’(2017)",Nov 25. 2020,이 글은 <월간 윤종신> 11월호의 ‘당신의 노래’ 코너에도 실려 있습니다. 좋아하...,윤종신/음악/음악평론
...,...,...,...,...,...
104,"가난, 그리고 비극",[어느 피란민 소녀의 슬픈 이야기],Aug 05. 2020,6.25 전쟁이 일어나자 우리 마을에는 북쪽에서 많은 피란민들이 내려와서 자리를 ...,"03화/미군과 우리 마을 사람들/04화/가난, 그리고 비극/현재글/05화/너무나 웃..."
105,<블루 발렌타인> 현실적이어서 더 슬픈 사랑 영화,영화 블루 발렌타인 줄거리 및 리뷰,Apr 06. 2020,곤히 자고 있는 신디를 깨우는 딘과 그들의 아이 프랭키. 일어나서 기껏 아침을 준비...,라이언고슬링/영화리뷰/영화
106,바흐의 대표곡 2,세상에서 가장 슬픈 음악,Apr 16. 2020,"오늘도 음악의 아버지, 바흐의 대표곡들을 소개해드리려고 합니다. (1편 복습하고 오...",바흐/클래식/바이올린
107,캄보디아의 슬픈 눈물 킬링필드,20세기 최악의 사건 Killing Fields,May 04. 2016,1950년 6.25 전쟁 때 캄보디아가 우리나라에 쌀을 지원했다는 사실을 알고 있는...,캄보디아/킬링필드/해외여행


In [144]:
# 저장된 파일들 합치기
data = pd.concat([sad_10, sad_20, sad_30, pd.DataFrame(diary)[['title', 'subtitle', 'date', 'content', 'list_keyword']]])
data = data.drop_duplicates()
data

,title,subtitle,date,content,list_keyword
0,아무것도가진 게없어 슬픈프롤레타리아여!,인천교육청 16차시 강의를 진행하며(10편),Aug 23. 2021,산업혁명은 ’혁명‘이란 용어를 사용할 정도로 당시 근대사회에 큰 충격과 변화를 이끌...,자본주의/경제공부/마르크스
1,아주 슬픈 시집,"- 『소월시 감상』, 박화목, 김원태 저, 1966년",Jul 11. 2021,"앞 뒤 표지가 다 떨어져 나가고 속만 남았다. 시간의 흔적이고 세월의 흔적이고, 또...",시집/문학/인문
2,비극의 원인을 규명하려는 인간의 슬픈 의지,"이태준《까마귀》(문학과지성사,2005)",Aug 15. 2021,이태준의 단편집 《까마귀》는 작가가 생명을 화두로 많은 고민을 해왔음을 보여준다...,이태준/까마귀/현대문학
3,슬픈 청령포,NaN,10시간전,청령포이름만으로도 가슴 에이는 슬픈 역사의 땅오랜만에 단종의 길을 돌아보려 영월에 ...,청령포/단종/영월
4,"대단히 슬픈 결말, <프리가이>",<프리가이>는 왜 <트루먼쇼>가 되지 못하는가,Aug 14. 2021,(치명적인 스포일러가 포함되어 있습니다.)<프리가이>의 결말은 대단히 슬프다. 이야...,영화/리뷰/에세이
...,...,...,...,...,...
48,이건 슬픈 영화다.,(Feat. 돈키호테 시대의 여성관이라니),May 27. 2019,-영화: <돈키호테를 죽인 사나이(The Man Who Killed Don Quix...,영화/영화감상/돈키호테
50,"여행에서 가장 슬픈 날, 마침표 : 뉴욕",언젠가 또다시 만날 날이 있겠지,Dec 19. 2016,이제 막 잠에서 깨어난 나를 향해 건너편 창가에 서 있던 줄리언이 말했다. 어두 ...,여행/뉴욕/공항
52,슬픈 순간 위에 내가 있었다,슬픔이 나를 만든다,Aug 06. 2020,"불빛 하나 없는 고요한 집에 외로이 발을 디딜 때면, 계기도 없는 슬픔을 맞이할 때...",슬픔/기쁨/취업
54,덧없는 욕망에 갇혀 슬픈 소리를 내는 옹졸한 삶 보다,,Aug 02. 2020,"El Condor pasa철새는 날아가고""달팽이가 되기보다는 참새가 되어야지그래, ...",잉카문명/소리/욕망


In [145]:
# 전체 데이터 저장
data.to_csv('슬픈_all.csv', encoding='utf-8-sig')